# Assignment 2

In [4]:

# Import necessary packages
import tarfile
import os
from bs4 import BeautifulSoup
import pandas as pd


In [1]:
file_path = "kungalv_slutpriser.tar.gz"

if file_path.endswith("tar.gz"):
    tar = tarfile.open(file_path, "r:gz")
    tar.extractall()
    tar.close()
elif file_path.endswith("tar"):
    tar = tarfile.open(file_path, "r:")
    tar.extractall()
    tar.close()



NameError: name 'tarfile' is not defined

In [5]:

with open("kungalv_slutpriser/kungalv_slutpris_page_01.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

for dateofsale in soup.find_all('span', {'class' : 'hcl-label hcl-label--state hcl-label--sold-at'}):
    print(dateofsale)
# date of sale
    
#address
    
#location
    
#area in the form (boarea + biarea)

#number of rooms
    
#area of the plot
    

#closing price


<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 9 oktober 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 5 oktober 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 3 oktober 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 2 oktober 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 27 september 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 26 september 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 25 september 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 22 september 2023
          </span>
<span class="hcl-label hcl-label--state hcl-label--sold-at">
            Såld 17 september 2023
    